In [ ]:
!pip install sklearn_crfsuite
import sklearn_crfsuite

In [ ]:
import rasa_nlu
import rasa_core
import warnings
warnings.filterwarnings('ignore')


print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))

In [ ]:
!pip install rasa_nlu
!pip install rasa_core

## 1. Natural Language Understanding 

### 1.1. Designing a happy path

### 1.2. Designing the NLU training data

In [ ]:
# Writing various intents with examples to nlu.md file

nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- how far
- hey there
- whats up
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- good by
- later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:thanks
- thanks
- thank you
- appreciated
- gracias


## intent:sleep
- when is bedtime
- when is naptime
- nap
- sleep
- tired
- sleepy

## intent: hungry
- i am hungry
- what time is lunch
- what time is breakfast
- what time is dinner
- lunch
- breakfast
- dinner
- snack time
- snack


## intent:schedule
- what's next
- what's in my schedule
- schedule
- do i have to do anything
- what's next now
- i don't know what to do


## intent:tasks
- what time is swimming
- when do i have swimming
- swimming 
- do i have to read
- reading
- what time do I have karate
- karate


 

"""


%store nlu_md >nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.



## 1.3 Designing the training pipeline



In [ ]:
configuration = """
language: "en"

pipeline:
- name: "tokenizer_whitespace"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_count_vectors"     # transform the sentence into a vector representation
- name: "intent_classifier_tensorflow_embedding"   # intent classifier
""" 

%store configuration > config.yml

Writing 'configuration' (str) to file 'config.yml'.


## 1.4 Training the first Rasa NLU Model


In [ ]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

def train_nlu_model():
    # loading the nlu training samples
    training_data = load_data("nlu.md")

    # trainer to educate our pipeline
    trainer = Trainer(config.load("config.yml"))

    # train the model!
    interpreter = trainer.train(training_data)

    # store it for future use
    model_directory = trainer.persist("./models/current", fixed_model_name="nlu")
    
    return interpreter, model_directory



Finally, let's train the model using the previously defined data and model configuration:

In [ ]:
interpreter, model_directory = train_nlu_model()

## Testing the model

We have trained the first version of our NLU model! Let's test it on various inputs:

In [ ]:
import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper function to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

#change the input message with your prefered inputs
input_message = "What time is swimming"
pprint(interpreter.parse(input_message))

## Handling out-of-scope inputs
When dealing with NLP models and AI, out-of-scope user inputs are very common challenge. These inputs represent the user requests which have nothing to do with the assistant's job. While it's very challenging to provide a sensible answer to each out-of-scope input, it's important to enable your assistant to identify such inputs and guide the user back to the conversation. First, let's enable our assistant to identify out-of-scope inputs. To do that, we will add a new intent called *out-of-scope* to our training dataset and provde some corresponding inputs:

In [ ]:
# Writing various intents with examples to nlu.md file

nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- how far
- hey there
- whats up
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- good by
- later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:thanks
- thanks
- thank you
- appreciated
- gracias


## intent:sleep
- when is bedtime
- when is naptime
- nap
- sleep
- tired
- sleepy

## intent: hungry
- i am hungry
- what time is lunch
- what time is breakfast
- what time is dinner
- lunch
- breakfast
- dinner
- snack time
- snack


## intent:schedule
- what's next
- what's in my schedule
- schedule
- do i have to do anything
- what's next now
- i don't know what to do


## intent:tasks
- what time is swimming
- when do i have swimming
- swimming 
- do i have to read
- reading
- what time do I have karate
- karate


## intent:out-of-scope
- I want pizza
- please help with my ice cream it's dripping
- no wait go back i want a dripping ice cream but a cone that catches it so you can drink the ice cream later
- i want a non dripping ice cream
- hey little mama let em whisper in your ear
- someone call the police i think the bot died
- show me a picture of a chicken
- neither
- I want french cuisine
- i am hungry
- restaurants
- restaurant
- you're a loser lmao
- can i be shown a gluten free restaurant
- i don't care!!!!
- i do not care how are you
- again?
- oh wait i gave you my work email address can i change it?
- hang on let me find it
- stop it, i do not care!!!
- really? you're so touchy?
- how come?
- I changed my mind
- what?
- did i break you

 

"""


%store nlu_md >nlu.md

In [ ]:
interpreter, model_directory = train_nlu_model()

In [ ]:
input_message = "I want pizza"
pprint(interpreter.parse(input_message))

## 1.6 Adding synonyms



In [ ]:
# Writing various intents with examples to nlu.md file

nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- how far
- hey there
- whats up
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- good by
- later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:thanks
- thanks
- thank you
- appreciated
- gracias


## intent:sleep
- when is bedtime
- when is naptime
- nap
- sleep
- tired
- sleepy

## intent: hungry
- i am hungry
- what time is lunch
- what time is breakfast
- what time is dinner
- lunch
- breakfast
- dinner
- snack time
- snack


## intent:schedule
- what's next
- what's in my schedule
- schedule
- do i have to do anything
- what's next now
- i don't know what to do


## intent:tasks
- what time is swimming
- when do i have swimming
- swimming 
- do i have to read
- reading
- what time do I have karate
- karate
- homework
- when do I have to do my homework


## intent:out-of-scope
- I want pizza
- please help with my ice cream it's dripping
- no wait go back i want a dripping ice cream but a cone that catches it so you can drink the ice cream later
- i want a non dripping ice cream
- hey little mama let em whisper in your ear
- someone call the police i think the bot died
- show me a picture of a chicken
- neither
- I want french cuisine
- i am hungry
- restaurants
- restaurant
- you're a loser lmao
- can i be shown a gluten free restaurant
- i don't care!!!!
- i do not care how are you
- again?
- oh wait i gave you my work email address can i change it?
- hang on let me find it
- stop it, i do not care!!!
- really? you're so touchy?
- how come?
- I changed my mind
- what?
- did i break you

 

"""


%store nlu_md >nlu.md

In [ ]:
configuration = """
language: "en"

pipeline:
- name: "tokenizer_whitespace"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_count_vectors"     # transform the sentence into a vector representation
- name: "intent_classifier_tensorflow_embedding"   # intent classifier
- name: "ner_synonyms"     #trains synonyms component
""" 

%store configuration > config.yml

In [ ]:
interpreter, model_directory = train_nlu_model()

In [ ]:
input_message = "Do I have to do my homework"
pprint(interpreter.parse(input_message))

In [ ]:
# Writing various intents with examples to nlu.md file

nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- how far
- hey there
- whats up
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- good by
- later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:thanks
- thanks
- thank you
- appreciated
- gracias


## intent:sleep
- when is bedtime
- when is naptime
- nap
- sleep
- tired
- sleepy

## intent: hungry
- i am hungry
- what time is lunch
- what time is breakfast
- what time is dinner
- lunch
- breakfast
- dinner
- snack time
- snack


## intent:schedule
- what's next
- what's in my schedule
- schedule
- do i have to do anything
- what's next now
- i don't know what to do


## intent:tasks
- what time is swimming
- when do i have swimming
- swimming 
- do i have to read
- reading
- what time do I have karate
- karate
- homework
- when do I have to do my homework
- when do I have my [task]
- what time is [task]
- do i have to [task]
- do i have to do [task]
- what time do I have [task]
- what time do I have [task] today
- what time do I have [task] tomorrow
- what time do I have [task] next week


## intent:out-of-scope
- I want pizza
- please help with my ice cream it's dripping
- no wait go back i want a dripping ice cream but a cone that catches it so you can drink the ice cream later
- i want a non dripping ice cream
- hey little mama let em whisper in your ear
- someone call the police i think the bot died
- show me a picture of a chicken
- neither
- I want french cuisine
- i am hungry
- restaurants
- restaurant
- you're a loser lmao
- can i be shown a gluten free restaurant
- i don't care!!!!
- i do not care how are you
- again?
- oh wait i gave you my work email address can i change it?
- hang on let me find it
- stop it, i do not care!!!
- really? you're so touchy?
- how come?
- I changed my mind
- what?
- did i break you

 

"""


%store nlu_md >nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


In [ ]:
configuration = """
language: "en"

pipeline:
- name: "tokenizer_whitespace"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_count_vectors"     # transform the sentence into a vector representation
- name: "intent_classifier_tensorflow_embedding"   # intent classifier
  intent_tokenization_flag: true # sets multi-intent tokenization
  intent_split_symbol: "+"       # sets which symbol should be used for tokenization
- name: "ner_synonyms"   # trains synonyms component 
""" 

%store configuration > config.yml

Writing 'configuration' (str) to file 'config.yml'.


In [ ]:
interpreter, model_directory = train_nlu_model()

In [ ]:
input_message = "what time do i have cooking today"
pprint(interpreter.parse(input_message))

## 1.8 Evaluating the NLU model



In [ ]:
from rasa_nlu.test import run_evaluation
import IPython
from IPython import display

run_evaluation("nlu.md", model_directory)

# 2. Dialogue Management


## 2.1 Designing the training stories

In [ ]:
stories_md = """
## happy path all info 1               
* greet              
  - utter_greet
* tasks
  - 
* length
  - utter_length
* thanks
  - utter_thanks
  
  
## happy path all info 2               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks    
  

## happy path all info 3               
* greet              
  - utter_greet
* speaker
  - utter_speaker
* length
  - utter_length
* thanks
  - utter_thanks  


## happy path no relevant audience            
* greet              
  - utter_greet
* recommend_session
  - utter_ask_relevant_audience
* inform{"relevant_audience":"Data Scientists"}
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks 

"""

%store stories_md > stories.md

## 2.2 Defining the domain



In [ ]:
domain_yml = """
intents:
- greet
- goodbye
- thanks
- sleep
- hungry
- schedule
- tasks
- out-of-scope




slots:
  tasks:
    type: unfeaturized
  time:
    type: unfeaturized
    



actions:
- utter_greet
- utter_thanks
- utter_goodbye
- utter_sleep
- utter_abstract
- utter_hungry
- utter_schedule
- utter_tasks
- utter_out_of_scope

templates:
  utter_greet:
    - text: "Hey! What's up"

  utter_thanks:
    - text: "You are very welcome!"

  utter_goodbye:
    - text: "See you later"
  
  utter_out_of_scope:
    - text: "Sorry, I can't help you with that"
  
  utter_schedule:
    - text: "Your test task is  {tasks} at {time}"
  
  utter_tasks:
    - text: "You have {tasks} at {time}"
    
  utter_sleep:
    - text: "Your next naptime is at {time}"

  utter_hungry:
    - text: "Your next mealtime is at {time}"

"""

%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


## 2.5 Training the dialogue model



In [ ]:
from rasa_core.policies import KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

def train_dialogue():
    # loading our neatly defined training dialogues
    agent = Agent("domain.yml", policies=[MemoizationPolicy(), KerasPolicy(epochs=200, max_history = 6)])
    training_data = agent.load_data('stories.md')


    agent.train(
        training_data)

    agent.persist('models/dialogue')


In [ ]:
train_dialogue()

## 2.6 Testing the dialogue model



In [ ]:
import IPython
from IPython.display import clear_output
from rasa_core.agent import Agent
from rasa_core.interpreter import NaturalLanguageInterpreter
from rasa_core.utils import EndpointConfig
import time

def load_assistant():
    messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
    interpreter = NaturalLanguageInterpreter.create(model_directory)
    endpoint = EndpointConfig('http://localhost:5055/webhook')
    agent = Agent.load('models/dialogue', interpreter=interpreter, action_endpoint = endpoint)

    print("Your bot is ready to talk! Type your messages here or send 'stop'")
    while True:
        a = input()
        if a == 'stop':
            break
        responses = agent.handle_text(a)
        for response in responses:
            print(response["text"])

In [ ]:
load_assistant()

## 2.7 Dialogue model evaluation

In [ ]:

!python -m rasa_core.evaluate --core models/dialogue --stories stories.md -o results
